In [2]:
from PIL import Image
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict
import sys
import time
import pickle
import torch
import torchvision.transforms.functional as tvf

from compressai.zoo.image import mbt2018_mean

torch.set_grad_enabled(False)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda:0')
print(torch.cuda.get_device_name(device))

NVIDIA GeForce RTX 4090


## Speed comparsion for CompressAI models

**Note:** this speed comparison includes the time to run neural network forward pass.

In [3]:
def test_on_kodak(model, verbose=None):
    model = model.to(device).eval()
    model.update(force=True)

    img_paths = list(Path('~/datasets/kodak').expanduser().rglob('*.png'))
    assert len(img_paths) == 24

    metrics = defaultdict(float)
    pbar = tqdm(img_paths, ascii=True, ncols=96, desc=f'{verbose}: ') if verbose else img_paths
    for impath in pbar:
        im = tvf.to_tensor(Image.open(impath).convert('RGB')).to(device).unsqueeze(0)

        enc_begin = time.time()
        compressed = model.compress(im)
        enc_end = time.time()
        outputs = model.decompress(compressed['strings'], compressed['shape'])
        dec_end = time.time()

        im_hat = outputs['x_hat']

        # encoding and decoding time
        metrics['enc-time'] += enc_end - enc_begin
        metrics['dec-time'] += dec_end - enc_end

        # estimate bpp and compute RGB PSNR
        num_bits = sys.getsizeof(pickle.dumps(compressed['strings'])) * 8
        bpp = num_bits / float(im.shape[2] * im.shape[3])
        psnr = -10.0 * (im - im_hat).square().mean().log10().item()
        # print(f'{impath.name}: PSNR={psnr:.2f}, BPP={bpp:.4f}')
        metrics['estimated-bpp'] += bpp
        metrics['rgb-psnr'] += psnr

    if verbose:
        for k, v in metrics.items():
            if 'time' in k:
                print(f'{k}: {v/24:.6f} s')
            else:
                print(f'{k}: {v/24:.4f}')


In [4]:
print('='*96)
model = mbt2018_mean(quality=1, pretrained=True)
test_on_kodak(model, verbose=None) # warmup
test_on_kodak(model, verbose='CompressAI')

print('='*96)
from compressai_models import MyGaussianConditional
model.gaussian_conditional = MyGaussianConditional(None)
test_on_kodak(model, verbose='This repo')


CompressAI: 100%|###############################################| 24/24 [00:01<00:00, 15.62it/s]


enc-time: 0.025475 s
dec-time: 0.028343 s
estimated-bpp: 0.1253
rgb-psnr: 27.6996


This repo: 100%|################################################| 24/24 [00:00<00:00, 24.90it/s]

enc-time: 0.010922 s
dec-time: 0.019145 s
estimated-bpp: 0.1253
rgb-psnr: 27.6996
